In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def minMaxNormalized(img):
    return cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX,dtype=cv2.CV_32F)

In [3]:
def high_boost(img, low_contrast, A):
    return ((A-1)*low_contrast + img)

In [30]:
# return red, green and blue freq. domain
def to_frequencyDomain(image): 
    '''Note: Don't forget the log1p so you can see the Magnitude'''
    normalized     = minMaxNormalized(image)
    norm_fft       = np.fft.fft2(normalized)
    norm_fft_shift = np.fft.fftshift(norm_fft)
    magnitude      = np.sqrt(norm_fft_shift.real ** 2 + norm_fft_shift.imag ** 2)
    return norm_fft_shift, magnitude

In [45]:
def to_spatialDomain(image):
    image_ishift = np.fft.ifftshift(image)
    image_ifft   = np.fft.ifft2(image_ishift)
    image_real   = np.abs(image_ifft)
    # denormlized  = image * 255
    return image_real

In [5]:
def reduceSaltAndPeperNoise():
    pass

In [6]:
def reduceGaussianNoise():
    pass

In [ ]:
#periodic noise...

In [7]:
# for one type of periodic noise. which type?
def periodicNoiseFilter(shape, radius, center_x, center_y):
    dimention1, dimention2 = shape
    mask = np.zeros((dimention1, dimention2, 2))

    for i in range(0, dimention1):
        for item in range(0, dimention2):
            radius_hat1 = np.sqrt((i - center_x) ** 2 + (item - center_y) ** 2)
            radius_hat2 = np.sqrt((i - dimention1 + center_x) ** 2 + (item - dimention2 + center_y) ** 2)

            if radius_hat1 <= radius or radius_hat2 <= radius:
                mask[i, item] = 0.0
            else:
                mask[i, item] = 1.0
    return mask

In [8]:
# maybe we need this function. negative of above function. return just periodic noise
def find_noise(shape, radius, center_x, center_y):
    dimention1, dimention2 = shape
    mask = np.zeros((dimention1, dimention2, 2))

    for i in range(0, dimention1):
        for item in range(0, dimention2):
            radius_hat1 = np.sqrt((i - center_x) ** 2 + (item - center_y) ** 2)
            radius_hat2 = np.sqrt((i - dimention1 + center_x) ** 2 + (item - dimention2 + center_y) ** 2)

            if radius_hat1 <= radius or radius_hat2 <= radius:
                mask[i, item] = 1.0
            else:
                mask[i, item] = 0.0
    return mask

In [9]:
# maybe we need this function
def psnr(original_image, reconstructed_image):
    original_image      = original_image.astype(np.float64)
    reconstructed_image = reconstructed_image.astype(np.float64)

    mse = np.mean((original_image - reconstructed_image) ** 2)

    if mse == 0:
        return float('inf')

    max_pixel_value = 255 
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

In [10]:
#input: BGR image
def equalizeHistForColorImages (img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)